In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v3/fact/granularity=monthly/

In [0]:

# creative_fact_url = "s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity={granularity}/date={date}/"
# adapp_fact_url = "s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity={granularity}/date={date}/"

# spark.read.parquet(adapp_fact_url.format(granularity=granularity, date=date_str)).createOrReplaceTempView("adapp")
# spark.read.parquet(creative_fact_url.format(granularity=granularity, date=date_str)).createOrReplaceTempView("creative")

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2019-03-31/").limit(1).show()

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2019-03-31/").limit(1).show()

In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2019-03-31/").createOrReplaceTempView("adapp")

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2019-03-31/").createOrReplaceTempView("creative")


# spark.sql("select * from (SELECT * FROM creative JOIN adapp ON creative.device_code = adapp.device_code AND creative.country_code = adapp.country_code AND creative.ad_platform_id = adapp.ad_platform_id AND creative.ad_app_id = adapp.ad_app_id WHERE creative.country_code != 'WW' AND creative.ad_platform_id != 1 and (creative.ad_platform_count is not null or creative.ad_platform_list is not null or creative.pub_app_count is not null or creative.pub_app_list is not null or creative.country_count is not null or creative.country_list is not null or creative.creative_share_of_ad_platform is not null or creative.creative_share_of_advertiser is not null)) as a")


# spark.sql("select *  from (SELECT adapp.device_code,adapp.country_code,adapp.ad_platform_id,adapp.ad_app_id,creative.creative_count,adapp.creative_id FROM creative JOIN adapp ON creative.device_code = adapp.device_code AND creative.country_code = adapp.country_code AND creative.ad_platform_id = adapp.ad_platform_id AND creative.ad_app_id = adapp.ad_app_id WHERE creative.country_code != 'WW' AND creative.ad_platform_id != 1 and (creative.ad_platform_count is not null or creative.ad_platform_list is not null or creative.pub_app_count is not null or creative.pub_app_list is not null or creative.country_count is not null or creative.country_list is not null or creative_share_of_ad_platform is not null or creative_share_of_advertiser is not null))  as a  group  by device_code, country_code,ad_platform_id,ad_app_id").show()
# (creative.ad_platform_count is not null or creative.ad_platform_list is not null or creative.pub_app_count is not null or creative.pub_app_list is not null or creative.country_count is not null or creative.country_list is not null or creative.creative_share_of_ad_platform is not null or creative.creative_share_of_advertiser is not null)

spark.sql("""select Sum(metric_creative_count)   as metric_creative_count, 
      Sum(distinct_creative_count) as distinct_creative_count 
from   (select device_code, 
              country_code, 
              ad_platform_id, 
              ad_app_id, 
              Max(creative_count)         as metric_creative_count, 
              Count(distinct creative_id) as distinct_creative_count 
        from   (select adapp.device_code, 
                      adapp.country_code, 
                      adapp.ad_platform_id, 
                      adapp.ad_app_id, 
                      adapp.creative_count, 
                      creative_id 
                from   creative 
                      join adapp 
                         on creative.device_code = adapp.device_code 
                            and creative.country_code = adapp.country_code 
                            and creative.ad_platform_id = adapp.ad_platform_id 
                            and creative.ad_app_id = adapp.ad_app_id 
                where  creative.country_code != 'WW' 
                      and creative.ad_platform_id != 1 
                      and ( creative.ad_platform_count is not null 
                              or creative.ad_platform_list is not null 
                              or creative.pub_app_count is not null 
                              or creative.pub_app_list is not null 
                              or creative.country_count is not null 
                              or creative.country_list is not null 
                              or creative_share_of_ad_platform is not 
                                 null 
                              or creative_share_of_advertiser is not 
                                 null )) 
              as a 
        group  by device_code, 
                  country_code, 
                  ad_platform_id, 
                  ad_app_id) as b""")
# 15585262

In [0]:

r = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2017-02-28/").filter("country_code='US' and device_code='ios-phone' and ad_app_id=1208907397 and ad_platform_id=101 and (ad_platform_count is not null or ad_platform_list is not null or pub_app_count is not null or pub_app_list is not null or country_count is not null or country_list is not null or creative_share_of_ad_platform is not null or creative_share_of_advertiser is not null)").select('creative_id').distinct().count()
print r
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2017-02-28/").filter("device_code='ios-phone' and ad_app_id=1208907397 and ad_platform_id=101 and country_code='US'").select("country_code","creative_count").show()





In [0]:

r = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2017-02-28/").filter("country_code='US' and device_code='ios-phone' and ad_app_id=994556645 and ad_platform_id=101").select('creative_id').distinct().count()
print r
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2017-02-28/").filter("device_code='ios-phone' and ad_platform_id=101 and country_code='US'").select("country_code","creative_count").show()





In [0]:

from pyspark.sql import functions as F
creative_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2017-02-28/").filter("country_code='US' and device_code='ios-phone' and ad_platform_id=101").groupBy("ad_app_id").agg(F.collect_set("creative_id").alias('creative_id_list'))

creative_df = creative_df.withColumn("len_creative", F.size("creative_id_list"))
print "creative_cout",creative_df.count()

ad_app_creative = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2017-02-28/").filter("device_code='ios-phone' and ad_platform_id=101 and country_code='US'").select("ad_app_id", "creative_count").distinct()

print "add_app_count",ad_app_creative.count()



merge_df = creative_df.join(ad_app_creative, ['ad_app_id'], how='inner')
print "merged_df count",merge_df.count()
merge_df.show()





In [0]:

from pyspark.sql import functions as F
creative_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2020-03-31/").groupBy("ad_app_id", "country_code", "ad_platform_id", "device_code").agg(F.collect_set("creative_id").alias('creative_id_list'))

creative_df = creative_df.withColumn("len_creative", F.size("creative_id_list"))
# print creative_df.filter("country_code !='WW' and ad_platform_id != 1").count()


# print "creative_cout",creative_df.count()

ad_app_creative = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2020-03-31/").select("ad_app_id", "country_code", "ad_platform_id", "device_code", "creative_count")

# print ad_app_creative.filter("country_code !='WW' and ad_platform_id != 1").count()
print "add_app_count",ad_app_creative.count()



merge_df = creative_df.join(ad_app_creative, ["ad_app_id", "country_code", "ad_platform_id", "device_code"], how='outer')
print "merged_df count",merge_df.count()
merge_df.show()





In [0]:

from pyspark.sql import functions as F
creative_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2017-02-28/").filter("country_code='US' and device_code='ios-phone' and ad_platform_id=101").groupBy("ad_app_id").agg(F.collect_set("creative_id").alias('creative_id_list'))

creative_df = creative_df.withColumn("len_creative", F.size("creative_id_list"))
print "creative_cout",creative_df.count()

ad_app_creative = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2017-02-28/").filter("device_code='ios-phone' and ad_platform_id=101 and country_code='US'").select("ad_app_id", "creative_count").distinct()

print "add_app_count",ad_app_creative.count()



merge_df = creative_df.join(ad_app_creative, ['ad_app_id'], how='inner')
print "merged_df count",merge_df.count()
merge_df.show()





In [0]:



spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2020-03-31/").filter("device_code='ios-phone' and ad_platform_id=325 and country_code='BR' and ad_app_id=284035177").select("creative_id").distinct().count()

In [0]:

from pyspark.sql import functions as F
creative_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2020-03-31/").groupBy("ad_app_id", "country_code", "ad_platform_id", "device_code").agg(F.collect_set("creative_id").alias('creative_id_list'))

creative_df = creative_df.withColumn("len_creative", F.size("creative_id_list"))
# print creative_df.filter("country_code !='WW' and ad_platform_id != 1").count()


print "creative_cout",creative_df.count()

ad_app_creative = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2020-03-31/").select("ad_app_id", "country_code", "ad_platform_id", "device_code", "creative_count")

# print ad_app_creative.filter("country_code !='WW' and ad_platform_id != 1").count()
print "add_app_count",ad_app_creative.count()



merge_df = creative_df.join(ad_app_creative, ["ad_app_id", "country_code", "ad_platform_id", "device_code"], how='outer')
print "merged_df count",merge_df.count()
merge_df.show()





In [0]:

from pyspark.sql import functions as F
creative_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=weekly/date=2019-12-28/").groupBy("ad_app_id", "country_code", "ad_platform_id", "device_code").agg(F.collect_set("creative_id").alias('creative_id_list'))

creative_df = creative_df.withColumn("creative_count", F.size("creative_id_list"))


# print "creative_cout",creative_df.count()

ad_app_creative = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=weekly/date=2019-12-28/").select("ad_app_id", "country_code", "ad_platform_id", "device_code", "creative_count")

# print "add_app_count",ad_app_creative.count()



merge_df = creative_df.alias("creative").join(ad_app_creative.alias("adapp"), [creative_df.device_code==ad_app_creative.device_code, creative_df.ad_platform_id==ad_app_creative.ad_platform_id, creative_df.country_code==ad_app_creative.country_code, creative_df.ad_app_id==ad_app_creative.ad_app_id,creative_df.creative_count != ad_app_creative.creative_count], how='outer')



merge_df.drop_duplicates().filter("adapp.ad_app_id is not null and creative.ad_app_id is not null").show()

print "merged_df count",merge_df.drop_duplicates().filter("adapp.ad_app_id is not null and creative.ad_app_id is not null").count()





In [0]:

print merge_df.drop_duplicates().count()
merge_df.drop_duplicates().filter("adapp.ad_app_id is not null and creative.ad_app_id is not null").show()

In [0]:

r = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2019-08-31/").filter("country_code='DE' and device_code='ios-phone' and ad_app_id=314498713 and ad_platform_id=324").select("creative_id").distinct().count()
print "creative:", r
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2019-08-31/").filter("country_code='DE' and device_code='ios-phone' and ad_app_id=314498713 and ad_platform_id=324").select("country_code","device_code", "ad_app_id", "ad_platform_id", "creative_count").show()

In [0]:
%%sh
PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF
select creative_count as adapp_creative_count from market.market_adapp_fact_v1 where 
            granularity='monthly' AND device_code = 'ios-phone' AND 
            country_code = 'DE' AND date='2019-08-31' AND ad_app_id=314498713 and ad_platform_id=324;




select count(distinct(creative_id)) as creative_count from  market.market_creative_adapp_fact_v1 where 
            granularity='monthly' AND  device_code = 'ios-phone' AND country_code ='DE' 
            AND date='2019-08-31' AND ad_app_id=314498713 and ad_platform_id=324;
EOF

In [0]:

r = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.creative-fact.v2/fact/granularity=monthly/date=2020-01-31/").filter("country_code='FI' and device_code='ios-tablet' and ad_app_id=359917414 and ad_platform_id=1").select("creative_id").distinct().count()
print "creative:", r
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-market/unified/market.adapp-fact.v2/fact/granularity=monthly/date=2020-01-31/").filter("country_code='FI' and device_code='ios-tablet' and ad_app_id=359917414 and ad_platform_id=1").select("country_code","device_code", "ad_app_id", "ad_platform_id", "creative_count").show()
